In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain beautifulsoup4 langchain_chroma

In [ ]:
pip install grandalf

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "your_langchain_key"
os.environ["LANGCHAIN_PROJECT"]="multi_query_rag"

In [ ]:
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass("OPENAI_API_KEY:")

OPENAI_API_KEY:··········


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


llm = ChatOpenAI(model_name="gpt-4o-mini")
parser = StrOutputParser()
emb = OpenAIEmbeddings()

In [ ]:
prompt_template = """You are an AI language model assistant. Your task is to generate three
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines. Original question: {question}"""
multi_prompt = ChatPromptTemplate.from_template(prompt_template)
multi_prompt

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate three\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question: {question}'))])

In [ ]:
mpromt_chain = multi_prompt | llm | parser
mprompts = mpromt_chain.invoke({"question": "What is the capital of France?"})
mprompts

"What city serves as the capital of France?  \nCan you tell me the name of France's capital city?  \nWhich city is recognized as the capital in France?  "

In [ ]:
queries = [q.strip() for q in mprompts.split("\n") if q.strip()]
queries

['What city serves as the capital of France?',
 "Can you tell me the name of France's capital city?",
 'Which city is recognized as the capital in France?']

In [ ]:
def convert_queries_to_list(mprompts):
    return [q.strip() for q in mprompts.split("\n") if q.strip()]

In [ ]:
mpromt_chain = mpromt_chain | convert_queries_to_list
mpromt_chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate three\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question: {question}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x798de8061ba0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x798de8063ca0>, root_client=<openai.OpenAI object at 0x798de8244c10>, root_async_client=<openai.AsyncOpenAI object at 0x798de8061c00>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()
| RunnableLa

In [ ]:
mpromt_chain.get_graph().print_ascii()                    # chain to generate multiple prompts

          +-------------+          
          | PromptInput |          
          +-------------+          
                 *                 
                 *                 
                 *                 
      +--------------------+       
      | ChatPromptTemplate |       
      +--------------------+       
                 *                 
                 *                 
                 *                 
          +------------+           
          | ChatOpenAI |           
          +------------+           
                 *                 
                 *                 
                 *                 
        +-----------------+        
        | StrOutputParser |        
        +-----------------+        
                 *                 
                 *                 
                 *                 
    +-------------------------+    
    | convert_queries_to_list |    
    +-------------------------+    
                 *          

In [ ]:
webpage = "https://towardsdatascience.com/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452"
loader = WebBaseLoader(webpage)
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=10)
all_splits = text_splitter.split_documents(data)
for i in range(len(all_splits)):
    print("Chunk ",i," : ",all_splits[i])

Chunk  0  :  page_content='Transformers Explained Visually (Part 1): Overview of Functionality | by Ketan Doshi | Towards Data ScienceOpen in appSign upSign inWriteSign upSign inINTUITIVE TRANSFORMERS SERIES NLPTransformers Explained Visually (Part 1): Overview of FunctionalityA Gentle Guide to Transformers, how they are used' metadata={'source': 'https://towardsdatascience.com/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452', 'title': 'Transformers Explained Visually (Part 1): Overview of Functionality | by Ketan Doshi | Towards Data Science', 'description': 'A Gentle Guide to Transformers, how they are used for NLP, and why they are better than RNNs, in Plain English. How Attention helps improve performance.', 'language': 'en'}
Chunk  1  :  page_content='are used for NLP, and why they are better than RNNs, in Plain English. How Attention helps improve performance.Ketan Doshi·FollowPublished inTowards Data Science·10 min read·Dec 13, 2020--23ListenSharePh

In [ ]:
multi_vectorstore = Chroma(
                          collection_name="multi_prompt_rag",
                          embedding_function=emb,
                          persist_directory="./multi_vectorstore")
multi_vectorstore.add_documents(all_splits)

<ipython-input-13-d9b64d3e4148>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  multi_vectorstore = Chroma(


['03328a76-3620-4a88-92b4-edd321357286',
 'ee3cc3c5-192f-4c5e-8769-e7e68888b40a',
 '2eedea41-cd29-47b3-ae25-c893fc82e8c7',
 '34fbe826-33be-4a97-bf54-af649a28e7be',
 '1bebb314-c0e9-4c2e-b8f6-8077329c4f6d',
 '6a05d67b-68b3-4a6f-83c5-c1812b0ad300',
 'e76ad752-549d-4bd3-9feb-8a6634e6031a',
 'c9273be5-8163-41a8-af49-f6d4dc866356',
 'feb7beec-10c7-4fbc-9502-3ef7154c339d',
 'fedd899f-b1bc-499e-8246-174b44110aff',
 '1e2d5119-b16c-4261-a038-06c60a6e45ed',
 'a798f30f-1f12-4b77-a38d-aa80f5dfb12b',
 '5ca3c61e-c9aa-4ba1-a009-f8d10a165d22',
 '612f2d0f-056d-4cc4-8c32-5e6fe8009f8d',
 '1e423d9d-c99e-4905-ad33-d817dae6db9b',
 '2794c800-fa39-4d09-8f3f-661058a4a35c',
 'a7870828-47e1-45aa-bbd5-1611c8b87c84',
 'c21d23af-19d4-4b5f-af1d-c8a9ccaa068f',
 '908303d1-8015-4180-a3e8-bc971340a13c',
 '1fbba878-05e6-4c04-9e11-d275c07e9c3b',
 'cc6db141-5069-4a57-bbad-71de34009cb8',
 'd0f129fa-9089-4c2b-9664-f4c9bd7381e7',
 '2f8f93d0-0c97-4812-b335-6256928a8e39',
 '2e9b7044-6b28-4cbc-8e49-68461e903e3a',
 '67176400-bd99-

In [ ]:
mrag_retriever = multi_vectorstore.as_retriever()
mrag_retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x798ddb68f280>)

In [ ]:
mrag_retriever_chain = mpromt_chain | mrag_retriever.map()
mrag_retriever_chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate three\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question: {question}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x798de8061ba0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x798de8063ca0>, root_client=<openai.OpenAI object at 0x798de8244c10>, root_async_client=<openai.AsyncOpenAI object at 0x798de8061c00>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()
| RunnableLa

In [ ]:
res = mrag_retriever_chain.invoke({"question": "Explain Transformer Classification architecture"})
res

[[Document(metadata={'description': 'A Gentle Guide to Transformers, how they are used for NLP, and why they are better than RNNs, in Plain English. How Attention helps improve performance.', 'language': 'en', 'source': 'https://towardsdatascience.com/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452', 'title': 'Transformers Explained Visually (Part 1): Overview of Functionality | by Ketan Doshi | Towards Data Science'}, page_content='and text classification. They are frequently used in sequence-to-sequence models for applications such as Machine Translation, Text Summarization, Question-Answering, Named Entity Recognition, and Speech Recognition.There are different flavors of the Transformer architecture for different problems.'),
  Document(metadata={'description': 'A Gentle Guide to Transformers, how they are used for NLP, and why they are better than RNNs, in Plain English. How Attention helps improve performance.', 'language': 'en', 'source': 'https://t

In [ ]:
def merge_docs(docs):
    merged_docs = []
    for doc_list in docs:
        merged_docs.extend(doc_list)
    return merged_docs

def remove_duplicates(docs):
    seen = set()
    unique_docs = []
    for doc in docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            unique_docs.append(doc)
    return unique_docs

In [ ]:
mprompt_final =  mrag_retriever_chain | merge_docs | remove_duplicates
mprompt_final

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='You are an AI language model assistant. Your task is to generate three\ndifferent versions of the given user question to retrieve relevant documents from a vector\ndatabase. By generating multiple perspectives on the user question, your goal is to help\nthe user overcome some of the limitations of the distance-based similarity search.\nProvide these alternative questions separated by newlines. Original question: {question}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x798de8061ba0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x798de8063ca0>, root_client=<openai.OpenAI object at 0x798de8244c10>, root_async_client=<openai.AsyncOpenAI object at 0x798de8061c00>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser()
| RunnableLa

In [ ]:
res = mprompt_final.invoke({"question": "Explain Transformer Classification architecture"})
res

[Document(metadata={'description': 'A Gentle Guide to Transformers, how they are used for NLP, and why they are better than RNNs, in Plain English. How Attention helps improve performance.', 'language': 'en', 'source': 'https://towardsdatascience.com/transformers-explained-visually-part-1-overview-of-functionality-95a6dd460452', 'title': 'Transformers Explained Visually (Part 1): Overview of Functionality | by Ketan Doshi | Towards Data Science'}, page_content='and text classification. They are frequently used in sequence-to-sequence models for applications such as Machine Translation, Text Summarization, Question-Answering, Named Entity Recognition, and Speech Recognition.There are different flavors of the Transformer architecture for different problems.')]

In [ ]:
mprompt_final.get_graph().print_ascii()

      +-------------+        
      | PromptInput |        
      +-------------+        
              *              
              *              
              *              
   +--------------------+    
   | ChatPromptTemplate |    
   +--------------------+    
              *              
              *              
              *              
       +------------+        
       | ChatOpenAI |        
       +------------+        
              *              
              *              
              *              
    +-----------------+      
    | StrOutputParser |      
    +-----------------+      
              *              
              *              
              *              
+-------------------------+  
| convert_queries_to_list |  
+-------------------------+  
              *              
              *              
              *              
  +----------------------+   
  | VectorStoreRetriever |   
  +----------------------+   
          

In [ ]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

multi_prompt_rag_chain = (
    {"context": mprompt_final, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [ ]:
multi_prompt_rag_chain.invoke({"question": "Explain Transformer Classification architecture"})

'The Transformer architecture is a powerful model used primarily in Natural Language Processing (NLP) tasks, including text classification. It operates based on a mechanism called "self-attention," which allows the model to weigh the significance of different words in a sentence, regardless of their position. This is in contrast to traditional models like Recurrent Neural Networks (RNNs), which process data sequentially and can struggle with long-range dependencies.\n\nIn the context of text classification, the Transformer architecture typically involves the following components:\n\n1. **Input Representation**: Text data is transformed into embeddings, which represent words or tokens in a continuous vector space. This often includes positional encodings to maintain the order of words.\n\n2. **Self-Attention Mechanism**: The self-attention layer allows the model to focus on different parts of the input sequence when making predictions. It calculates attention scores to determine how muc

In [ ]:
multi_prompt_rag_chain.get_graph().print_ascii()

               +---------------------------------+           
               | Parallel<context,question>Input |           
               +---------------------------------+           
                      ****               ****                
                   ***                       ***             
                 **                             ***          
  +--------------------+                           **        
  | ChatPromptTemplate |                            *        
  +--------------------+                            *        
              *                                     *        
              *                                     *        
              *                                     *        
      +------------+                                *        
      | ChatOpenAI |                                *        
      +------------+                                *        
              *                                     *        
        